In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator # type: ignore
from datetime import datetime
import pandas as pd

    
def transform():
    df = pd.read_csv('/tmp/extracted_data.csv')
    df['name'] = df['name'].str.upper()
    df.to_csv('/tmp/transformed_data.csv', index=False)

def load():
    df = pd.read_csv('/tmp/transformed_data.csv')
    df.to_csv('/tmp/final_data.csv', index=False)
    print("Data loaded:", df)

with DAG(
    dag_id='etl_example',
    start_date=datetime(2023, 1, 1),
    # schedule_interval='@daily',
    schedule='@daily',
    catchup=False,
    tags=['example', 'etl']
) as dag:
    def extract():
        df = pd.read_csv('https://raw.githubusercontent.com/microsoft/DataStoriesSamples/refs/heads/master/samples/FraudDetectionOnADL/Data/transactions.csv')
        print(df)
        return df
    
    t1 = PythonOperator(
        task_id='extract',
        python_callable=extract
    )

    t2 = PythonOperator(
        task_id='transform',
        python_callable=transform
    )

    t3 = PythonOperator(
        task_id='load',
        python_callable=load
    )